In [1]:
import functools

import requests

year = 2023
day = 12

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open("cookie.txt", "r") as file:
    session = file.read()

headers = {"cookie": f"session={session}"}

with open(f"inputs/day_{str(day).zfill(2)}_input.txt", "wb") as file:
    file.write(requests.get(url, headers=headers).content)

lines = [x.strip() for x in open(f"inputs/day_{str(day).zfill(2)}_input.txt", "r")]
print(lines[:5])

['.???#??????#. 6,1', '???##???##?#??#?#..# 1,14,1', '#??????#????#?###?# 3,1,10,1', '.???????#????. 1,2,1,2', '?##?.??.???.. 3,2,2']


In [56]:
import re

test = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".split("\n")

source = lines

#print(test)

UNK = '?'
DMG = '#'
OPR = '.'

def valid(springs, counts):
    collapsed_string = re.sub(r'\.+', '.', springs)
    act_counts = [len(x) for x in collapsed_string.split('.') if not x == '']
    if act_counts == counts:
        return True
    else:
        return False
    
    
def binary(num, length=32):
    return format(num, '#0{}b'.format(length + 2))
    
from tqdm import tqdm
valid_counts_arr = []
for row in tqdm(source):
    springs, counts  = row.split()
    counts = [int(x) for x in counts.split(",")]
    
    # springs *= 5
    # n_counts = []
    # for i in range(0,5):
    #     n_counts += counts
    # counts = n_counts
    
    num_unk = springs.count("?")
    num_perms = 2**num_unk
    
    valid_counts = 0
    for i in range(0, num_perms):
        new_springs = springs
        bin_i = list(reversed(bin(i)[2:]))
        for j in range(0, num_unk):
            if j < len(bin_i) and bin_i[j] == '1':
                new_springs = new_springs.replace("?","#",1)
            else:
                new_springs = new_springs.replace("?",".",1)        
    
        if valid(new_springs, counts):
            valid_counts += 1
            
    valid_counts_arr.append(valid_counts)
    
#print(valid_counts_arr)
print(sum(valid_counts_arr))


100%|██████████| 1000/1000 [00:57<00:00, 17.30it/s]

7599


In [60]:
from tqdm import tqdm

test = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".split("\n")

@functools.lru_cache(maxsize=None)
def ck(springs, counts, c, g_i, c_group):
    
    #print(springs, counts, f"c:{c}, g_i:{g_i}, c_group:{c_group}, {len(counts)}")

    key = (c, g_i, c_group)
    if key in DP.keys():
        return DP[key]

    #if you're at the end
    if len(springs) == c:
        # either there are groups left
        if g_i < len(counts):
            #print("bad", springs, counts, f"c:{c}, g_i:{g_i}, c_group:{c_group}, {len(counts)}")
            
            if g_i  == len(counts) - 1 and c_group == counts[g_i]:
               # print("good", springs, counts, f"c:{c}, g_i:{g_i}, c_group:{c_group}, {len(counts)}")
                return 1
            
            return 0
        # already finished last group
        elif g_i == len(counts):
            if c_group > 0:
                # if it was the right length, keep going.
                if c_group == counts[g_i]:
                   # print("good", springs, counts, f"c:{c}, g_i:{g_i}, c_group:{c_group}, {len(counts)}")
                    return 1
                else:
                    return 0
            else:
                return 1
        else:
            return 0
        

    
    total = 0
    
    # If unknown, repeat with both.
    # if springs[c] == '?':
    #     total += ck(springs[:c] + '.' + springs[c+1:], counts, c, g_i, c_group)
    #     total += ck(springs[:c] + '#' + springs[c+1:], counts, c, g_i, c_group)
    # If it's a space.
    if springs[c] == '.' or springs[c] == '?':
        # If you're in a spring already. finish that spring.
        if c_group > 0:
            # if it was the right length, keep going.
            if c_group == counts[g_i]:
                total += ck(springs, counts, c+1, g_i+1, 0)
            # else:
            #     return 0
        # Otherwise, just keep going.
        else:
            total += ck(springs, counts, c+1, g_i, 0)
    # If it's part of a spring
    if springs[c] == '#' or springs[c] == '?':
        # If you're not in a spring, start one
        if c_group == 0:
            # but only if there's one left
            if g_i != len(counts):
                total += ck(springs, counts, c+1, g_i, 1)
        # otherwise, just keep going
        else:
            total += ck(springs, counts, c+1, g_i, c_group+1)

    if key in DP:
        assert False

    if key not in DP.keys():
        DP[key] = total    
    return total
    
valid_counts_arr = []
for row in lines:
    springs, counts  = row.split()
    counts = tuple(int(x) for x in counts.split(","))
    
    n_counts = []
    n_springs = ""
    for i in range(0,5):
        n_springs += springs + "?"
        n_counts += counts
    counts = tuple(n_counts)
    springs = n_springs[:-1]
    
    DP = {}
    
    perm = ck(springs, counts, 0, 0, 0)
    valid_counts_arr.append(perm)
    
    DP.clear()
        
print(valid_counts_arr)
print(sum(valid_counts_arr))

[1024, 32, 32, 48417448, 1024, 1, 70307500, 1, 10093140, 141158875, 3483, 7776, 1362234, 4918, 32, 293358, 26244, 162, 62166742, 3078251, 162, 151047661, 32, 124416, 280, 3149348613, 14426000, 62307, 2811072, 1024, 12630808, 1266576, 153242, 8794, 30000, 3888, 9604, 81, 1, 1, 1, 1, 2322495, 1911752, 7203, 512, 158358, 512, 243, 39366, 272, 506250, 60000, 1, 5715, 1250, 2592, 516384, 162, 598385492, 19683, 32, 243, 32, 1537063, 32, 32768, 394737138148, 243, 768, 17621, 11899700525790, 3103296, 32, 9604, 189536, 212154, 707650, 200438869, 5184, 135134330, 32805, 16, 2328612, 1, 39366, 2592, 2592, 5184, 5419456, 32, 1126800, 1750983488, 1, 14384, 2413607, 1, 1035693, 32, 768, 297914064, 1, 460699808, 512, 70002, 7274, 4948, 512, 124416, 768, 2135274, 3483900, 12304348786, 7776, 60000, 10502270, 165473835, 19521461338, 246966190, 16557, 1256298000, 68, 1024, 1, 16384, 39366, 32, 7776, 768, 243, 1, 249519, 1, 256, 79161, 162, 210007, 7776, 159618, 525574964, 117128, 1, 235126, 921477, 25923